# Galopp-Preprocessor

For this step in the project I will use jupyter notebook, because it is way faster to visualize and more flexible on doin data preprocessing. (The exploratory data analysis part will also be with jupyter notebook)

In [ ]:
print("Preprocessing...")

### Imports

In [2]:
import pandas as pd
import numpy as np

### Load dataset

In [3]:
galopp = pd.read_csv("csvs/all_races.csv")
galopp.sample(5)

,Date,Location,Distance,Prize,Category,Class,Ground_state,Horses,Unnamed: 8,m,€,Unnamed: 11,Unnamed: 12,Boden:,[]
1635,08. Juli 2014,Hamburg,1600 m,0 €,NaN,NaN,Boden: g-w,"[' 1. ', ' Easte...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3568,09. November 2015,Mons,2300 m,8000 €,D,NaN,Boden: Sand ...,"[' 1. ', ' Maste...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6620,05. Juli 2018,Hamburg,1600 m,8000 €,F,IV,Boden: gut,"[' 1. ', ' Dark ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
502,14. Juli 2013,München,1400 m,0 €,NaN,NaN,Boden: gut,"[' 1. ', ' Lord ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3067,21. Juli 2015,Bad Harzburg,2000 m,8000 €,D,III,Boden: gut,"[' 1. ', ' Korad...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Look at general information

In [4]:
galopp.columns

Index(['Date', 'Location', 'Distance', 'Prize', 'Category', 'Class',
       'Ground_state', 'Horses', 'Unnamed: 8', ' m  ', '   €', 'Unnamed: 11',
       'Unnamed: 12', '                     Boden:                   ', '[]'],
      dtype='object')

In [5]:
print(galopp.isna().sum())
print("")
print(galopp.isna().sum()/len(galopp)*100)

Date                                                15
Location                                            15
Distance                                             0
Prize                                                0
Category                                          3115
Class                                             4245
Ground_state                                         0
Horses                                               0
Unnamed: 8                                        9218
 m                                                9218
   €                                              9218
Unnamed: 11                                       9218
Unnamed: 12                                       9218
                     Boden:                       9218
[]                                                9218
dtype: int64

Date                                                0.162725
Location                                            0.162725
Distance                               

In [6]:
galopp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9218 entries, 0 to 9217
Data columns (total 15 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Date                                            9203 non-null   object 
 1   Location                                        9203 non-null   object 
 2   Distance                                        9218 non-null   object 
 3   Prize                                           9218 non-null   object 
 4   Category                                        6103 non-null   object 
 5   Class                                           4973 non-null   object 
 6   Ground_state                                    9218 non-null   object 
 7   Horses                                          9218 non-null   object 
 8   Unnamed: 8                                      0 non-null      float64
 9    m                                       

First thing I see is that there are some NaN values. 15 each on _Date_ and _Location_ (which are only 0.16% of the whole set). Because only a few are missing, and they are not really important to my goal, I drop the corresponding rows.

Also, the columns 'Category' and 'Class' are missing 3115 and 4245, which are ~33% and ~46%. Normally, because they are categorical, they could be filled with the mode, but so many values are missing, so I decide to drop them, as they are not important for my goal.

Next, the datatypes should be changed for _Distance_ and _Prize_ to int. But before that, the 'm' in _Distance_ and the '€' in _Prize_ have to be removed. (Also, there are '\xa0's in both columns for each entry! These have to be removed too before converting to int.). And, at last, fill the empty strings with a 0, otherwise this would lead to ValueErrors when converting to int.

The _Ground_state_ always contains the prefix 'Boden: ', this can also be removed.

In [7]:
# Drop Rows with Date and Location NaN
galopp = galopp[galopp["Date"].notna()]
galopp = galopp[galopp["Location"].notna()]

# Remove units
galopp["Distance"] = galopp["Distance"].apply(lambda x: x.replace("m", ""))
galopp["Prize"] = galopp["Prize"].apply(lambda x: x.replace("€", ""))

# Remove bytes
galopp["Distance"] = galopp["Distance"].apply(lambda x: x.replace("\xa0", ""))
galopp["Prize"] = galopp["Prize"].apply(lambda x: x.replace("\xa0", ""))

# Fill empty strings with 0
galopp["Distance"] = galopp["Distance"].apply(lambda x: "0" if len(x) == 0 else x)
galopp["Prize"] = galopp["Prize"].apply(lambda x: "0" if len(x) == 0 else x)

# Change datatype to int
galopp["Distance"] = galopp["Distance"].astype(int)
galopp["Prize"] = galopp["Prize"].astype(int)

# Remove 'Boden: ' prefix
galopp["Ground_state"] = galopp["Ground_state"].apply(lambda x: x.replace("Boden: ", ""))
galopp["Ground_state"] = galopp["Ground_state"].apply(lambda x: x.strip())

# Drop columns
galopp.drop(columns=["Category", "Class"], inplace=True)
galopp = galopp.iloc[:, :-7]

Looking at the dataframe again:

In [8]:
galopp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9203 entries, 0 to 9217
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Date          9203 non-null   object
 1   Location      9203 non-null   object
 2   Distance      9203 non-null   int64 
 3   Prize         9203 non-null   int64 
 4   Ground_state  9203 non-null   object
 5   Horses        9203 non-null   object
dtypes: int64(2), object(4)
memory usage: 503.3+ KB


In [9]:
galopp.sample(10)

,Date,Location,Distance,Prize,Ground_state,Horses
1716,26. Juli 2014,Bad Harzburg,1850,0,g-w,"[' 1. ', ' Ipos ..."
5195,21. Mai 2017,Köln,1850,5100,gut,"[' 1. ', ' Meerw..."
9184,31. Oktober 2020,Dortmund (Sand),1800,3000,nass,"[' 1. ', ' Tauru..."
5665,13. August 2017,Hoppegarten,1800,7500,gut,"[' 1. ', ' Chris..."
2875,14. Juni 2015,Köln,1600,8750,gut,"[' 1. ', ' Fame ..."
1851,23. August 2014,Leipzig,1600,0,gut,"[' 1. ', ' Franc..."
1633,08. Juli 2014,Hamburg,1400,0,g-w,"[' 1. ', ' Milli..."
7904,28. Juli 2019,München,1600,0,gut,"[' 1. ', ' Angry..."
1067,19. März 2014,Neuss (Sand),1900,0,trocken,"[' 1. ', ' Dream..."
2342,05. Januar 2015,Mons,2100,5600,Sand,"[' 1. ', ' Blue ..."


No NaNs and the fitting datatype, also the samples looking good aswell.
So for those columns everything that needs to be done is done. Lets go on with the horses per race. For this, I intend to:
- get the list of horses and make another dataframe of it
- Clean this dataset (No column names and datatypes here)
- Return the cleaned dataframe as a list and replace it
- Save each horse participation in another dataframe / csv

### Clean general info about the race

In [10]:
def clean_date_string(x):
    x = x.strip()
    x = x.replace(". ", "/")
    x = x.replace(" ", "/")
    x = x.replace("Januar", "01")
    x = x.replace("Februar", "02")
    x = x.replace("März", "03")
    x = x.replace("April", "04")
    x = x.replace("Mai", "05")
    x = x.replace("Juni", "06")
    x = x.replace("Juli", "07")
    x = x.replace("August", "08")
    x = x.replace("September", "09")
    x = x.replace("Oktober", "10")
    x = x.replace("November", "11")
    x = x.replace("Dezember", "12")
    return x

def clean_location_string(x):
    return x.lower().strip()

def clean_ground_state_string(x):
    return x.lower().strip()

galopp["Date"] = galopp["Date"].apply(clean_date_string)
galopp["Location"] = galopp["Location"].apply(clean_location_string)
galopp["Ground_state"] = galopp["Ground_state"].apply(clean_ground_state_string)

### Clean races and generate a participants dataframe

In [11]:
def clean_placement_string(x):
    
    if "NS" in x: # Treat "Nichtstarter", horses who didn't start the race
        x = -1
    else:
        x = x.replace(".","")
        x = x.replace("'","")
        x = x.replace("[","")
        x = x.replace("]", "")
        x = x.strip()
    
    return x

def clean_horse_name_string(x):
    x = x.replace("'", "")
    x = x.strip()
    x = x.lower()
    return x

def clean_jockey_name_string(x):
    x = x.replace("'", "")
    x = x.strip()
    x = x.lower()
    
    if "." in x:
        while "." in x:
            x = x[x.index(".", )+1:] # Get surname by dot
    elif len(x.split()) == 2:
        x = x.split()[1]  # Get surname when both names are in the name string
    else:
        pass
    
    return x

def clean_trainer_name_string(x):
    x = x.replace("'", "")
    x = x.strip()
    x = x.lower()
    
    if "." in x:
        while "." in x:
            x = x[x.index(".", )+1:] # Get surname by dot
    elif len(x.split()) == 2:
        x = x.split()[1]  # Get surname when both names are in the name string
    else:
        x=x
    
    return x

def clean_weight_string(x):
    x = x.replace("'", "")
    x = x.replace(",",".")
    x = x.replace("]", "")
    x = x.strip()
    return x

In [12]:
# Load, clean, and replace each race
races = []
horses = []
columns = ["Place", "Horse_name", "Jockey_name", "Trainer_name", "Weight"]

for index in range(len(galopp)):
    row = galopp.iloc[index, :]
    
    # Load row as a seperate dataset and make it a dataframe for easier editing
    split = row["Horses"].split(", ")
    try:
        row_reshaped = np.array(split).reshape((-1, 5))
        race_df = pd.DataFrame(data=row_reshaped, columns=columns)
        # Clean dataset (and save a version with the races)
        race_df["Place"] = race_df["Place"].apply(clean_placement_string)
        race_df["Horse_name"] = race_df["Horse_name"].apply(clean_horse_name_string)
        race_df["Jockey_name"] = race_df["Jockey_name"].apply(clean_jockey_name_string)
        race_df["Trainer_name"] = race_df["Trainer_name"].apply(clean_trainer_name_string)
        #print("ek")
        race_df["Weight"] = race_df["Weight"].apply(clean_weight_string)
        
        #Get normal informatio of the race
        race_info = galopp.iloc[index, :5]
        
        # Add each participant and info about the race itself to a list
        for index_2 in range(len(race_df)):
            horses.append(pd.concat([race_df.iloc[index_2], race_info], axis=0).values)

        # Add the whole race to a list
        #races.append(galopp.iloc[index, :].tolist())
        # TODO TODO TODO
        
    except:
        races.append("DELETE THIS ROW") # Some rows just dont fit... delete the rows afterwards!
    
print(len(horses))
# Save participations for further inspection
all_participations_df = pd.DataFrame(data=horses, columns=columns+["Date", "Location", "Distance", "Prize", "Ground_state"])
all_participations_df.to_csv("csvs/participations.csv", index=False)

84332


In [13]:
# Replace the cleaned races with the old races in the dataframe
#galopp = pd.DataFrame(np.array(races).reshape((9218, 4)), columns=["Distance", "Prize", "Ground_state", "Horses"])
#galopp.to_csv("csvs/galopp_cleaned.csv", index=False)